<a href="https://colab.research.google.com/github/ykim879/whats.todays.trend/blob/master/news_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Analyzer

In [5]:
import requests
import pandas as pd

In [25]:
response = requests.get('https://newsapi.org/v2/sources?'
			   'language=en&country=us&'
			   'apiKey=55b9f473b6e642ac862d315a124c6619')
sources = response.json()['sources']
df = pd.DataFrame();
pd.set_option('display.max_columns', 7)
for source in sources:
  row = pd.Series(source)
  df = df.append(row, ignore_index = True)
df.set_index("name", inplace = True)

In [22]:
print(df)

                                category country  \
id                                                 
abc-news                         general      us   
al-jazeera-english               general      us   
ars-technica                  technology      us   
associated-press                 general      us   
axios                            general      us   
bleacher-report                   sports      us   
bloomberg                       business      us   
breitbart-news                   general      us   
business-insider                business      us   
buzzfeed                   entertainment      us   
cbs-news                         general      us   
cnn                              general      us   
crypto-coins-news             technology      us   
engadget                      technology      us   
entertainment-weekly       entertainment      us   
espn                              sports      us   
espn-cric-info                    sports      us   
fortune     

## Method: eachNewsHeadlines
### collects each 55 news headline

In [35]:
def eachNewsHeadlines(ID):
	response = requests.get('https://newsapi.org/v2/top-headlines?sources=' + ID + '&apiKey=55b9f473b6e642ac862d315a124c6619')
	json = response.json()
	if json['status'] != 'ok':
		raise IOError('API key is probably overused :(')
	topNews = []
	topNews.append(ID)
	for elem in json['articles']:
		topNews.append(elem['title'])
	return topNews

In [45]:
headlines = []
for id in df['id']:
  headlines.append(eachNewsHeadlines(id))

['abc-news', 'Protesters arrested in Charlotte as GOP delegates arrive', 'Trump ordered to pay $44,100 in Stormy Daniels legal fees', 'US WeChat users sue Trump over order banning messaging app', 'House set to vote on USPS bill amid mail-in voting controversy', 'Officials: Police shoot, kill man outside store in Louisiana', 'Statue of lauded rocker Chris Cornell vandalized in Seattle', "Family says USPS lost veterans' remains", 'Tropical Storm Laura, Marco simultaneously heading toward Gulf of Mexico', 'Smoke fills air as Portland police clear riot at precinct', "Steve Bannon calls his arrest 'a political hit job,' says effort was in support of Trump"]
['al-jazeera-english', 'West African mediators arrive in Mali to push for coup reversal', "Alexei Navalny's health condition 'very worrying'", 'Global coronavirus death toll hits 800,000: Live news']
['ars-technica', 'New Wonder Woman 1984 trailer offers first look at Kristen Wiig’s Cheetah', 'Hint of COVID-19 immunity: 3 sailors with an

## Method: parseWords
### this will parse the headlines to count how many times each vocabs appears in headlines

In [37]:
import re

In [51]:
def parseWords(headlines, dic):
  for headline in headlines:
    for line in headline[1:]:
      vocabs = re.split('\W+', line)
      for vocab in vocabs:
        if vocab in dic:
          dic[vocab] += 1
        else:
          dic[vocab] = 1

In [52]:
parseVocab = {}
parseWords(headlines, parseVocab)
print(parseVocab)

{'Protesters': 3, 'arrested': 1, 'in': 83, 'Charlotte': 1, 'as': 19, 'GOP': 16, 'delegates': 1, 'arrive': 2, 'Trump': 54, 'ordered': 2, 'to': 122, 'pay': 3, '44': 2, '100': 7, 'Stormy': 2, 'Daniels': 2, 'legal': 2, 'fees': 2, 'US': 8, 'WeChat': 1, 'users': 2, 'sue': 3, 'over': 9, 'order': 1, 'banning': 1, 'messaging': 1, 'app': 3, 'House': 15, 'set': 3, 'vote': 9, 'on': 47, 'USPS': 8, 'bill': 3, 'amid': 5, 'mail': 5, 'voting': 3, 'controversy': 1, 'Officials': 3, 'Police': 5, 'shoot': 1, 'kill': 1, 'man': 1, 'outside': 1, 'store': 1, 'Louisiana': 1, 'Statue': 1, 'of': 58, 'lauded': 1, 'rocker': 1, 'Chris': 2, 'Cornell': 1, 'vandalized': 1, 'Seattle': 1, 'Family': 1, 'says': 11, 'lost': 1, 'veterans': 1, 'remains': 1, 'Tropical': 2, 'Storm': 1, 'Laura': 2, 'Marco': 2, 'simultaneously': 1, 'heading': 1, 'toward': 1, 'Gulf': 6, 'Mexico': 2, 'Smoke': 2, 'fills': 1, 'air': 1, 'Portland': 5, 'police': 1, 'clear': 1, 'riot': 1, 'at': 13, 'precinct': 1, 'Steve': 2, 'Bannon': 1, 'calls': 5, 'hi